# Imports / Configuration

In [63]:
import numpy as np

import sys,os

data_path = os.getcwd()

try:
    import localgraphclustering as lgc
except:
    # when the package is not installed, import the local version instead. 
    # the notebook must be placed in the original "notebooks/" folder
    sys.path.append("../")
    import localgraphclustering as lgc

## Load dataset

In [64]:
g = lgc.GraphLocal(os.path.join(data_path,'datasets/JohnsHopkins.graphml'),'graphml')

/usr/local/lib/python3.7/site-packages/localgraphclustering/GraphLocal.py:217: UserWarning:

Loading a graphml is not efficient, we suggest using an edgelist format for this API.



## Define proximal gradient descent function

In [65]:
def proximal_gradient_descent(A, d, ref_node, rho, alpha, eps, stepsize=1.):
    
    # Number of nodes in the graph
    n = d.shape[0]
    
    # Initialize seed vector
    seed = np.zeros(n)
    seed[ref_node] = 1
    
    # Initialized paramters
    x = np.zeros(n)

    # Initiliaze algorithm statistics
    err = 100.
    ite = 0

    # The algorithm starts here
    while err > eps:
        # Store old parameters
        x_old = x
        # Compute gradient
        grad = ((1+alpha)/2)*(d*x)  - ((1-alpha)/2)*(A@x) - alpha * seed
        # Update parameters using a gradient step
        x = x - stepsize * grad / d 
        # Update parameters using the proximal step
        x = np.sign(x) * np.maximum(abs(x) - stepsize * rho * alpha, 0)
        # Compute termination criterion
        err = np.linalg.norm(x_old - x, np.inf)
        # Increase iteration count
        ite += 1
        
    return x

## Run proximal gradient descent on the givne graph

In [69]:
# L1-regularization parameter for the spectral problem
rho = 8.0e-5
# Teleportation parameter of the PageRank model
alpha = 0.1
# Refence node
ref_node = 1

# Matrices and vectors that are needed by the present implementation of proximal gradient descent
A = g.adjacency_matrix
d = g.d

# Stepsize for algorithm
stepsize = (1+alpha)/2
# Termination tolerance
epsilon = 1.0e-10

# Call proximal gradient descent
x = proximal_gradient_descent(A, d, ref_node, rho, alpha, epsilon)

## Print output

In [70]:
x

array([0.        , 0.00377886, 0.        , ..., 0.        , 0.        ,
       0.        ])

## Sparsity of the output

In [71]:
np.count_nonzero(x)/g._num_vertices

0.010083381811130502